# NTP Server Change Tool

Super simple tool here guys. In the YAML section, specify your routers & switches where you'd like this change to be made, and add the NTP servers you want pushed out. There's also a section with some login variables for you to set. Once you've done that, go ahead and run the tool.

## Login Credentials

In [ ]:
user = 'autobot'
password = 'juniper123'

## System & NTP Server Config (YAML)

In [ ]:
setup_yaml = """
---
hosts:
    - router1
    - switch1
    - switch2

servers:
    - 10.0.1.20
    - 10.0.1.40
"""

## Nothing to Change Below Here - Output Follows

In [ ]:
from jnpr.junos import Device
from jnpr.junos.utils.config import Config
from jnpr.junos.exception import *
import yaml
from jinja2 import Template
import sys

# Prepare Jinja2 Template
ntp_servers_template = """
system {
    ntp {
        {% for item in servers %}
        server {{ item }};
        {% endfor %}
    }
}
"""
j2_template = Template(ntp_servers_template)

# Load up YAML Config
config = yaml.load(setup_yaml)
hosts = config.get('hosts')

for host in hosts:
    dev = Device(host=host, user=user, password=password)

    # Open Connection
    try:
        dev.open()
    except Exception as err:
        print("Cannot connect to device: {}".format(err))
        sys.exit(0)

    dev.bind(cu=Config)

    # Lock the configuration, load changes, commit
    print("Locking the configuration on: {}".format(host))
    try:
        dev.cu.lock()
    except LockError:
        print("Error: Unable to lock configuration on: {}".format(host))
        dev.close()
        sys.exit(0)

    print("Loading configuration changes on: {}".format(host))
    try:
        # If your devices don't already have NTP configured, comment out
        # the next 4 lines
        set_commands = """
        delete system ntp
        """
        dev.cu.load(set_commands, format='set')
        dev.cu.load(template=j2_template,
                    template_vars=config, format='text', merge=True)
    except ValueError as err:
        print(err.message)

    print("Committing the configuration on: {}".format(host))
    try:
        dev.cu.commit()
        # print dev.cu.diff()
    except CommitError:
        print("Error: Unable to commit configuration")
        print("Unlocking the configuration")
        try:
            dev.cu.unlock()
        except UnlockError:
            print("Error: Unable to unlock configuration")
        dev.close()
        sys.exit(0)

    print("Unlocking the configuration")
    try:
        dev.cu.unlock()
    except UnlockError:
        print("Error: Unable to unlock configuration")

    dev.close()
    print("-------------------------------------------")